# Problem-Specific Coordinate Generation for HyperNEAT Substrates

## Setup

The testing was done on a 16GB VRAM GPU with CUDA 12.8. VRAM usage is determined mainly by substrate and population size.

### Imports

This setup requires some dependencies, mainly TensorNEAT, JAX, numpy, matplotlib, NetworkX, scikit-learn and wandb for logging. Using a virtual environment (i.e. conda) is highly recommended. Python 3.10.18 was used in development and testing.

In [1]:
import jax
import wandb

from config import config
from tensorneat.algorithm.hyperneat import FullSubstrate

from substrate_generation.pca_coor_generator import PCAanalyzer 
from substrate_generation.fa_coor_generator import FactorAnalyzer
from substrate_generation.random_coor_generator import RandomCoordinateGenerator
from substrate_generation.data_sampling import sample_from_pretrained_agent
from evol_pipeline.brax_env import CustomBraxEnv
from evol_pipeline.custom_pipeline import CustomPipeline
from substrate_generation.hidden_layers import HiddenLayerGenerator
from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import save_coordinates_to_csv, setup_folder_structure
from evol_pipeline.evol_algorithm import create_evol_algorithm

A quick setup of the folder structure to avoid errors further down the line.

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir"]
setup_folder_structure(OUTPUT_DIR)

### Setup Environment

[Brax environments](https://github.com/google/brax/tree/main/brax/envs) are used for this experiment through the [TensorNEAT wrapper](https://github.com/EMI-Group/tensorneat/tree/main/src/tensorneat/problem/rl).

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=False,
    sample_episodes=16,
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]

print("env_problem.input_shape: ", env_problem.input_shape)
print("env_problem.input_shape: ", env_problem.output_shape)

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-09-23 18:54:39.109415: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-09-23 18:54:49.647006: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

env_problem.input_shape:  (27,)
env_problem.input_shape:  (8,)


### Random Substrate

We also want to test if our data-driven approaches are better than arbritarily chosen coordinates in an equally high-dimensional space. For this purpose we generate a substrate with the given number of dimensions in which all coordinates are random binary combinations, e.g. (0,1,0,1,1,0).

In [4]:
random_generator = RandomCoordinateGenerator(
    obs_size=obs_size,
    act_size=act_size,
    feature_dims=11,
    depth_factor=config["substrate"]["depth_factor"],
    width_factor=config["substrate"]["width_factor"],
)
input_coors, output_coors = random_generator.generate_io_coordinates()

Generating random coordinates with 11 feature dimensions...
Using binary alphabet [0, 1] for 27 coordinates.
Using binary alphabet [0, 1] for 8 coordinates.
Added layering dimension. Final coordinate size: 12


## Substrate Generation

Now it's time to generate the full substrates. The input and ouput nodes have already been generated above, so the hidden layers are the crucial part that's missing.

### Generating Hidden Layers

A `HiddenLayerGenerator` class is used to generate hidden nodes from the input node. There are of course, many ways these hidden nodes can be defined in the high-dimensional space. The class has a few options hard-coded:

* `one_hot`: This does a one-hot encoding of the dimensions, so there will be one coordinate for each dimension. E.g. for 3 Dimensions: (1,0,0),(0,1,0),(0,0,1)
* `one_double_hot`: This is the same as one-hot, but has two coordinates in every dimension, e.g. for 3 dimensions: (1,0,0),(0,1,0),(0,0,1),(2,0,0),(0,2,0),(0,0,2)
* `two_hot`: This adds all possible pairs of dimensions as coordinates to the one_hor ecoding, e.g. for 3 dimensions: (1,0,0),(0,1,0),(0,0,1),(1,1,0),(1,0,1),(0,1,1)
* `shift`: This schema takes all input nodes and simply "shifts" every one of them into the layering direction by 1 for each layer.
* `shift_two`: This does the same as `shift`, but doubles the number of nodes by adding duplicates where all the directions are multiplied by 2. This is like treating it as a vector and simply doubling it.

One important thing to notice is that the number of generated hidden nodes is highly dependent on the number of dimensions (one_hot/one_double_hot linear and two_hot exponential), except for the `shift` and `shift_two` methods.

In [5]:
hidden_layer_gen = HiddenLayerGenerator(
    env_name=env_name,
    obs_size=obs_size,
    act_size=act_size,
    hidden_layer_type=config["substrate"]["hidden_layer_type"],
    hidden_depth=config["substrate"]["hidden_depth"],
    depth_factor=config["substrate"]["depth_factor"],
    width_factor=config["substrate"]["width_factor"],
)

hidden_coors = hidden_layer_gen.get_hidden_coors(input_coors=input_coors)

active_substrate = FullSubstrate(
    input_coors=input_coors,
    hidden_coors=hidden_coors,
    output_coors=output_coors,
)

## Neuroevolution

Finally the substrates are ready to be used for neuroevolution with HyperNEAT. Weights and Biases (wandb) is used for logging and the CPPNs with highest fitness and their phenotypes are visualized. Coordinates are logged for further analysis.


In [ ]:
for i in range(4):

    evol_algorithm = create_evol_algorithm(substrate=active_substrate)

    initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])
    print("Intial CPPN Layers:", initial_cppn_layers)
    substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)

    wanbd_name = f"{env_name}_cycle{i+1}_{substrate_dimensions}d"
    wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, "cycles"]

    wandb.init(
        name=wanbd_name,
        project="substrate_dims",
        tags=wandb_tags,
        config=config  
    )

    wandb.config.update(
        {
            "substrate": {
                "obs_size": obs_size,
                "act_size": act_size,
                "num_queries": active_substrate.query_coors.shape[0],
                "query_dim": active_substrate.query_coors.shape[1],
                },
            "algorithm": {
                "neat": {
                    "num_inputs": evol_algorithm.num_inputs,
                    },
                "genome": {
                    "cppn_init_hidden_layers": initial_cppn_layers,
                    },
                },
        },
    )

    pipeline = CustomPipeline(
        algorithm=evol_algorithm,
        problem=env_problem,
        seed=config["experiment"]["seed"],
        generation_limit=config["pipeline"]["generation_limit"],
        fitness_target=config["pipeline"]["fitness_target"],
        is_save=False,
        save_dir=config["experiment"]["output_dir"],
    )

    init_state = pipeline.setup()
    state = pipeline.auto_run(state=init_state)

    print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

    wandb.finish()

    state_for_show = state[0] if isinstance(state, tuple) else state

    # Transform the best genome into network parameters
    best_genome = pipeline.best_genome

    # Built-in show method to produce and save a video of the agent
    pipeline.show(
        state=state_for_show,
        best=best_genome,
        output_type="mp4",
        save_path=f"{OUTPUT_DIR}/video/agent_cycle{i+1}.mp4",
    )

    # Visualizes the CPPN
    visualize_cppn(
        pipeline=pipeline, 
        state=state, 
        save_path=f"{OUTPUT_DIR}/topology/cppn_cycle{i+1}.svg"
        )
    # Visualizes a representation of the neural network in 2D space
    visualize_nn(
        pipeline=pipeline, 
        state=state, 
        save_path=f"{OUTPUT_DIR}/topology/nn_cycle{i+1}.svg", 
        substrate=active_substrate, 
        input_coors=input_coors, 
        hidden_coors=hidden_coors, 
        output_coors=output_coors,
        hidden_depth=config["substrate"]["hidden_depth"], 
        hidden_depth=config["substrate"]["hidden_depth"], 
        max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
        )
    
    # all input and output coordinates are logged for further analysis
    log_coors = input_coors
    for coor in hidden_coors:
        log_coors.append(coor)
    for coor in output_coors:
        log_coors.append(coor)
    save_coordinates_to_csv(
        coordinates=log_coors,
        filepath=f"{OUTPUT_DIR}/coordinates/cycle{i+1}_io.csv",
    )

    key = jax.random.PRNGKey(config["experiment"]["seed"]) # Use seed from config
    sampled_expert_data = sample_from_pretrained_agent(key, state, best_genome, pipeline, env_problem, 1000)

    analyzer_fa = FactorAnalyzer(
        data=sampled_expert_data, 
        obs_size=obs_size, 
        act_size=act_size,
        feature_dims=11-(i*2),
        hidden_depth=config["substrate"]["hidden_depth"],
        width_factor=config["substrate"]["width_factor"],
        normalize_coors=config["data_analysis"]["normalize_coors"],
        depth_factor=config["substrate"]["depth_factor"],
    )
    input_coors, output_coors = analyzer_fa.generate_io_coordinates()
    analyzer_fa.plot_factor_loadings(save_path=f"{OUTPUT_DIR}/data_analysis/fa_heatmap__cycle{i+1}.png")
    hidden_coors = hidden_layer_gen.get_hidden_coors(input_coors=input_coors)

    active_substrate = FullSubstrate(
    input_coors=input_coors,
    hidden_coors=hidden_coors,
    output_coors=output_coors,
    )

Intial CPPN Layers: [12, 4]


initializing
initializing finished
start compile
compile finished, cost time: 33.186222s
Generation: 1, Cost time: 9554.39ms
 	fitness: valid cnt: 1000, max: 52.3155, min: -117.6875, mean: -29.0639, std: 54.9360

	node counts: max: 42, min: 40, mean: 41.06
 	conn counts: max: 342, min: 311, mean: 338.89
 	species: 50, [11, 28, 35, 34, 11, 5, 5, 18, 15, 25, 10, 22, 7, 10, 2, 13, 14, 23, 10, 11, 8, 30, 5, 14, 11, 3, 14, 14, 7, 7, 8, 19, 6, 20, 15, 16, 9, 26, 16, 11, 15, 10, 6, 3, 10, 23, 13, 15, 3, 334]

Generation: 2, Cost time: 9600.94ms
 	fitness: valid cnt: 1000, max: 58.8745, min: -112.5500, mean: 31.9107, std: 34.2569

	node counts: max: 43, min: 39, mean: 41.11
 	conn counts: max: 343, min: 284, mean: 338.67
 	species: 50, [18, 2, 9, 11, 1, 2, 8, 73, 31, 1, 1, 1, 1, 7, 16, 26, 3, 2, 16, 13, 1, 18, 21, 15, 16, 1, 4, 21, 2, 6, 5, 18, 13, 20, 14, 40, 7, 4, 46, 7, 5, 5, 6, 5, 18, 1, 4, 3, 4, 428]

Generation: 3, Cost time: 9497.77ms
 	fitness: valid cnt: 1000, max: 53.3813, min: -115.

compute_ms,▆▃▅▂▄▅▁▁▂▂▂▁▄▂▁▂▂▂▂▆▅▂▂▂▆▆▇▃▃▃▃▅▃▇▇▇█▇██
compute_ms_pop,▁▅▅▂▃▂▃▁▁▅▄▅▂▂▂▃▃▂▂▃▆▂▃▄▄▃▃▆▃▃▇▃▃▄▇██▇▆█
fitness_max,▁▁▁▂▃▄▃▃▅▅▆▆▇▇▇▇▆▇▇▇▆██▇▇▇▇▇▇▇█▇██▇█▇▇▇█
fitness_mean,▁▁▁▁▂▃▃▄▄▄▆▆▅▄▅▆▆▆▆▅▆▆▇▆▆▆▅▅▅▆▆▆▆▆██▅▆▆▆
fitness_min,▇▇▇▇▇▇▇▇▇█▇▇▇▇▅▇▅▅▇█▃▇▇▆▇▅▂▇▁▇▇▆▇▆▇▄▇▇▁▇
fitness_std,▁▂▂▂▃▅▅▆▅▆▆▆▇▇▇▆▇▆▆▆▆▆▇▇▆▇▇▇▆▇▇▇▇▇▆█▇▇▇▇
generation,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
compute_ms,9611.88483
compute_ms_pop,9.61188
fitness_max,680.99603
fitness_mean,97.40757


2025-09-23 20:14:36.301350: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-09-23 20:14:36.301365: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


Total reward:  787.3771


/home/andi/anaconda3/envs/jax/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


mp4 saved to:  output/ant/video/agent_cycle1.mp4
Visualizing CPPN. Saving to output/ant/topology/cppn_cycle1.svg.


KeyError: 'conn_weight_lower_bound'